In [1]:
admission_df = spark.read.csv("../../mimic3/data/ADMISSIONS.csv", header=True, mode="DROPMALFORMED")

In [2]:
admission_df.createOrReplaceTempView("admission")

diabetes = spark.sql("SELECT * FROM admission WHERE DIAGNOSIS like '%DIABETES%'")
lungcancer = spark.sql("SELECT * FROM admission WHERE DIAGNOSIS like '%LUNG CANCER%'")
kidney = spark.sql("SELECT * FROM admission WHERE DIAGNOSIS like '%KIDNEY%'")


In [5]:
diabetes.take(2)

[Row(ROW_ID=u'1683', SUBJECT_ID=u'1363', HADM_ID=u'163367', ADMITTIME=u'2166-02-23 20:56:00', DISCHTIME=u'2166-02-28 10:45:00', DEATHTIME=None, ADMISSION_TYPE=u'URGENT', ADMISSION_LOCATION=u'TRANSFER FROM HOSP/EXTRAM', DISCHARGE_LOCATION=u'HOME', INSURANCE=u'Private', LANGUAGE=u'ENGL', RELIGION=u'NOT SPECIFIED', MARITAL_STATUS=u'MARRIED', ETHNICITY=u'WHITE', EDREGTIME=None, EDOUTTIME=None, DIAGNOSIS=u'DIABETES;HYPERGLYCEMIA', HOSPITAL_EXPIRE_FLAG=u'0', HAS_CHARTEVENTS_DATA=u'1'),
 Row(ROW_ID=u'2838', SUBJECT_ID=u'2345', HADM_ID=u'169351', ADMITTIME=u'2103-08-11 01:05:00', DISCHTIME=u'2103-08-22 19:53:00', DEATHTIME=None, ADMISSION_TYPE=u'ELECTIVE', ADMISSION_LOCATION=u'PHYS REFERRAL/NORMAL DELI', DISCHARGE_LOCATION=u'HOME', INSURANCE=u'Private', LANGUAGE=None, RELIGION=u'PROTESTANT QUAKER', MARITAL_STATUS=u'MARRIED', ETHNICITY=u'WHITE', EDREGTIME=None, EDOUTTIME=None, DIAGNOSIS=u'DIABETES MELLITUS-PRE-OP PANCREAS TRANSPLANT', HOSPITAL_EXPIRE_FLAG=u'0', HAS_CHARTEVENTS_DATA=u'1')]

In [3]:
#get subject_id from Diabetes patients
dia_id = diabetes.select("SUBJECT_ID").rdd.flatMap(lambda x: x).collect()
#exlude Diabetes patients in lung cancer patients
lungcancer = lungcancer.filter(~lungcancer.SUBJECT_ID.isin(dia_id))
#exlude Diabetes patients in kidney patients
kidney = kidney.filter(~kidney.SUBJECT_ID.isin(dia_id))

In [4]:
diabetes.count()+lungcancer.count()+kidney.count()

384

In [5]:
kidney_id = kidney.select("SUBJECT_ID").rdd.flatMap(lambda x: x).collect()
#exlude kidney patients in lung cancer patients
lungcancer = lungcancer.filter(~lungcancer.SUBJECT_ID.isin(kidney_id))

In [6]:
diabetes.count()+lungcancer.count()+kidney.count()

384

In [7]:
from pyspark.sql.functions import lit
#add new feature "disease" to each row
diabetes = diabetes.withColumn('disease', lit(0))
lungcancer = lungcancer.withColumn('disease', lit(1))
kidney = kidney.withColumn('disease', lit(2))
#add new feature "disease_desc" for each row
diabetes = diabetes.withColumn('disease_desc', lit("Diabetes"))
lungcancer = lungcancer.withColumn('disease_desc', lit("Lung Cancer"))
kidney = kidney.withColumn('disease_desc', lit("Kidney Condition"))

In [8]:
diabetes.count()+lungcancer.count()+kidney.count()

384

In [9]:
#union three dataframe
dataset = diabetes.union(lungcancer)
dataset = dataset.union(kidney)

In [15]:
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

wSpec = Window.partitionBy("SUBJECT_ID").orderBy("ROW_ID")

rankData = dataset.select('SUBJECT_ID','ADMISSION_TYPE','ADMISSION_LOCATION', 
                          'DISCHARGE_LOCATION','INSURANCE','RELIGION',
                          'LANGUAGE','MARITAL_STATUS','ETHNICITY','DIAGNOSIS','disease','disease_desc', 
                          rank().over(wSpec).alias("rowNum"))

In [16]:
distinctData = rankData[rankData.rowNum == 1]
#get subject_id from Diabetes distinctData
distinctData_id = distinctData.select("SUBJECT_ID").rdd.flatMap(lambda x: x).collect()

In [26]:
from pyspark.sql.functions import col
all_patiets_group = admission_df.groupby('SUBJECT_ID').count()
distinct_patiets_group = all_patiets_group.filter(all_patiets_group.SUBJECT_ID.isin(distinctData_id))\
                         .select(col('SUBJECT_ID').alias('patient_id'),col('count').alias('frequency'))

In [27]:
finalData = distinctData.join(distinct_patiets_group, 
                  distinct_patiets_group.patient_id == distinctData.SUBJECT_ID).select(
                          'SUBJECT_ID','ADMISSION_TYPE','ADMISSION_LOCATION', 
                          'DISCHARGE_LOCATION','INSURANCE','RELIGION',
                          'LANGUAGE','MARITAL_STATUS','ETHNICITY','frequency',
                          'DIAGNOSIS','disease','disease_desc')

In [28]:
finalData.printSchema()

root
 |-- SUBJECT_ID: string (nullable = true)
 |-- ADMISSION_TYPE: string (nullable = true)
 |-- ADMISSION_LOCATION: string (nullable = true)
 |-- DISCHARGE_LOCATION: string (nullable = true)
 |-- INSURANCE: string (nullable = true)
 |-- RELIGION: string (nullable = true)
 |-- LANGUAGE: string (nullable = true)
 |-- MARITAL_STATUS: string (nullable = true)
 |-- ETHNICITY: string (nullable = true)
 |-- frequency: long (nullable = false)
 |-- DIAGNOSIS: string (nullable = true)
 |-- disease: integer (nullable = false)
 |-- disease_desc: string (nullable = false)



Fill all null value with 'NoneValue'.

In [29]:
finalData = finalData.na.fill('NoneValue')

Transform Spark Dataframe to Pandas Dataframe.

In [30]:
pd_df = finalData.toPandas()

In [31]:
pd_df.to_csv('data/mimic3_demographic_data.csv', index=False) 

Select all categorical variables from pandas dataframe.

In [23]:
cat_df = pd_df.iloc[:,1:10]

In [24]:
cat_df.head()

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,RELIGION,LANGUAGE,MARITAL_STATUS,ETHNICITY,frequncy
0,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Medicare,PROTESTANT QUAKER,ENGL,WIDOWED,WHITE,1
1,ELECTIVE,PHYS REFERRAL/NORMAL DELI,SNF,Private,NOT SPECIFIED,ENGL,WIDOWED,WHITE,1
2,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,LONG TERM CARE HOSPITAL,Medicaid,PROTESTANT QUAKER,SPAN,SINGLE,HISPANIC OR LATINO,1
3,ELECTIVE,PHYS REFERRAL/NORMAL DELI,HOME HEALTH CARE,Private,OTHER,ENGL,MARRIED,WHITE,1
4,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DISCH-TRAN TO PSYCH HOSP,Medicare,CATHOLIC,NoneValue,SINGLE,BLACK/AFRICAN AMERICAN,1


In [25]:
from sklearn import preprocessing

# TODO: create a LabelEncoder object and fit it to each feature in cat_df


# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()


# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
cat_df_2 = cat_df.apply(le.fit_transform)
cat_df_2.head()

,ADMISSION_TYPE,ADMISSION_LOCATION,DISCHARGE_LOCATION,INSURANCE,RELIGION,LANGUAGE,MARITAL_STATUS,ETHNICITY,frequncy
0,0,2,11,2,13,4,6,13,0
1,0,2,11,3,10,4,6,13,0
2,1,3,8,1,13,13,5,7,0
3,0,2,4,3,12,4,2,13,0
4,1,3,2,2,2,8,5,4,0


In [71]:
# TODO: create a OneHotEncoder object, and fit it to all of  cat_df

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(cat_df_2)

# 3. Transform
onehotlabels = enc.transform(cat_df_2)
onehotlabels.shape

(372, 89)

In [73]:
print onehotlabels

  (0, 79)	1.0
  (0, 76)	1.0
  (0, 62)	1.0
  (0, 46)	1.0
  (0, 39)	1.0
  (0, 23)	1.0
  (0, 20)	1.0
  (0, 5)	1.0
  (0, 0)	1.0
  (1, 79)	1.0
  (1, 76)	1.0
  (1, 62)	1.0
  (1, 46)	1.0
  (1, 36)	1.0
  (1, 24)	1.0
  (1, 20)	1.0
  (1, 5)	1.0
  (1, 0)	1.0
  (2, 79)	1.0
  (2, 70)	1.0
  (2, 61)	1.0
  (2, 55)	1.0
  (2, 39)	1.0
  (2, 22)	1.0
  (2, 17)	1.0
  :	:
  (369, 56)	1.0
  (369, 46)	1.0
  (369, 28)	1.0
  (369, 24)	1.0
  (369, 20)	1.0
  (369, 3)	1.0
  (369, 1)	1.0
  (370, 80)	1.0
  (370, 67)	1.0
  (370, 61)	1.0
  (370, 46)	1.0
  (370, 39)	1.0
  (370, 23)	1.0
  (370, 17)	1.0
  (370, 6)	1.0
  (370, 1)	1.0
  (371, 79)	1.0
  (371, 65)	1.0
  (371, 58)	1.0
  (371, 43)	1.0
  (371, 32)	1.0
  (371, 23)	1.0
  (371, 13)	1.0
  (371, 5)	1.0
  (371, 1)	1.0
